In [ ]:
import numpy as np
import pandas as pd
import shap
import seaborn as sns
import warnings
np.random.seed(10)
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager as fm, rcParams
# # plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签
# # plt.rcParams['axes.unicode_minus']=False
# # plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

In [ ]:
path = './ESCC/dataset/GEO_5y_gene.csv'

In [ ]:
raw_data = pd.read_csv(path)
print(raw_data.shape)
raw_data.head()

In [ ]:
raw_data.info()

In [ ]:
columns_to_drop = ['Unnamed: 0']
# 删除指定的列
raw_data = raw_data.drop(columns=columns_to_drop)
raw_data.head()

In [ ]:
# heatmap 
def heatmap(data, method='pearson', camp='RdYlGn', figsize=(10 ,8)):
    #     mask = np.zeros_like(df2.corr())
    #     mask[np.tril_indices_from(mask)] = True
    plt.figure(figsize=figsize, dpi= 80)
    sns.heatmap(data.corr(method=method), \
                xticklabels=data.corr(method=method).columns, \
                yticklabels=data.corr(method=method).columns, cmap=camp, \
                center=0, annot=True)

In [ ]:
heatmap(data=train_data, method='spearman', figsize=(25,20))

In [ ]:
train_data = raw_data.copy()

In [ ]:
data_X = train_data.drop(['OS'], axis=1)
data_bin_y = train_data['OS']
data_bin_y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_bin_y, test_bin_y = train_test_split(data_X,data_bin_y,test_size=0.2) #,random_state=2022
print(train_x.shape)
print(test_x.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

test_bin_y.value_counts()

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [7]:
from sklearn.linear_model import LogisticRegression
param_grid = {'C': [0.1, 1, 2, 5]}
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=3)
grid_search.fit(train_x, train_bin_y)
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_lr = grid_search.best_estimator_
lr_y_pred = best_lr.predict(test_x)     # lr_y_pred 
print(classification_report(test_bin_y,lr_y_pred)) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [2, 4, 6, 8, 10],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 3, 5, 10]
}

RF = RandomForestClassifier(random_state=2022)  
grid_search = GridSearchCV(estimator=RF, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(train_x, train_bin_y)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_rf = grid_search.best_estimator_

rf_y_pred = best_rf.predict(test_x)
print(classification_report(test_bin_y, rf_y_pred))

In [ ]:
import xgboost as xgb
param_grid = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [50, 80, 100, 200],
    'subsample': [0.5, 0.6, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.5, 0.6, 0.8, 0.9, 1.0]
}

xgb_model = xgb.XGBClassifier(use_label_encoder=False)  # random_state=2022, 
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(train_x, train_bin_y)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_xgb = grid_search.best_estimator_

# 预测并评估模型
xgb_y_pred = best_xgb.predict(test_x)
print(classification_report(test_bin_y, xgb_y_pred))

In [ ]:
roc_auc_score(test_bin_y,xgb_y_pred)

In [ ]:
from sklearn import svm 
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

svm_model = svm.SVC(probability=True, random_state=2022)
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(train_x, train_bin_y)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_svm = grid_search.best_estimator_
best_svm.fit(train_x, train_bin_y)

svm_y_pred = best_svm.predict(test_x)
print(classification_report(test_bin_y, svm_y_pred))

In [ ]:
from sklearn import tree
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 4,6,8],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [2, 4, 6, 8]
}

dt_model = tree.DecisionTreeClassifier(random_state=2022)
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=4, n_jobs=-1, scoring='accuracy')
grid_search.fit(train_x, train_bin_y)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_dt = grid_search.best_estimator_
best_dt.fit(train_x, train_bin_y)

dt_y_pred = best_dt.predict(test_x)
print(classification_report(test_bin_y, dt_y_pred))

In [ ]:
from sklearn.linear_model import Lasso
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]
}

lasso_model = Lasso(random_state=2022)
grid_search = GridSearchCV(estimator=lasso_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(train_x, train_bin_y)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_lasso = grid_search.best_estimator_
best_lasso.fit(train_x, train_bin_y)

la_y_pred = best_lasso.predict(test_x)

threshold = 0.5
la_y_pred_class = np.where(la_y_pred > threshold, 1, 0)

print(classification_report(test_bin_y, la_y_pred_class))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_model = KNeighborsClassifier()
grid_search = GridSearchCV(estimator=knn_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(train_x, train_bin_y)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_knn = grid_search.best_estimator_
best_knn.fit(train_x, train_bin_y)

knn_y_pred = best_knn.predict(test_x)
print(classification_report(test_bin_y, knn_y_pred))

In [ ]:
# SHAP
explainer = shap.KernelExplainer(best_knn.predict_proba, train_x)

In [ ]:
shap_values_test = explainer.shap_values(test_x)
shap_values_train = explainer.shap_values(train_x)

In [ ]:
# SHAP Beeswarm Plot
shap.summary_plot(shap_values, test_x, plot_type="bar")

In [ ]:
plt.rcParams['figure.dpi'] = 100 
shap.summary_plot(shap_values_train[1], train_x)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {
    'n_estimators': [100, 400, 600, 700],
    'learning_rate': [0.008, 0.01, 0.012],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 3, 4]
}

gbdt = GradientBoostingClassifier()
grid_search = GridSearchCV(estimator=gbdt, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
grid_search.fit(train_x, train_bin_y)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_gbdt = grid_search.best_estimator_

gbdt_y_pred = best_gbdt.predict(test_x)
print(classification_report(test_bin_y, gbdt_y_pred))

In [ ]:
roc_auc_score(test_bin_y, gbdt_y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
lr_auc = roc_auc_score(test_bin_y,lr_y_pred)
rf_auc = roc_auc_score(test_bin_y,rf_y_pred)
xgb_auc = roc_auc_score(test_bin_y,xgb_y_pred)
svm_auc = roc_auc_score(test_bin_y,svm_y_pred)
dt_auc = roc_auc_score(test_bin_y,dt_y_pred)
la_auc = roc_auc_score(test_bin_y, la_y_pred_class)
knn_auc = roc_auc_score(test_bin_y, knn_y_pred)
gbdt_auc = roc_auc_score(test_bin_y, gbdt_y_pred)
print('AUC area:')
print('LR = %0.4f' % lr_auc)
print('RF = %0.4f' % rf_auc)
print('XGB = %0.4f' % xgb_auc)
print('SVM = %0.4f' % svm_auc)
print('DT = %0.4f' % dt_auc)

print('lasso = %0.4f' % la_auc)
print('knn = %0.4f' % knn_auc)
print('gbdt = %0.4f' % gbdt_auc)

In [ ]:
fpr_1,tpr_1, thresholds_1 = roc_curve(test_bin_y,best_lr.predict_proba(test_x)[:,1])  # lr
fpr_2,tpr_2, thresholds_2 = roc_curve(test_bin_y,best_rf.predict_proba(test_x)[:,1])  # rf
fpr_3,tpr_3, thresholds_3 = roc_curve(test_bin_y,best_xgb.predict_proba(test_x)[:,1])  # xgb
fpr_4,tpr_4, thresholds_4 = roc_curve(test_bin_y,best_svm.decision_function(test_x)) # svm
fpr_5,tpr_5, thresholds_5 = roc_curve(test_bin_y,best_dt.predict_proba(test_x)[:,1]) # dt

fpr_6,tpr_6, thresholds_6 = roc_curve(test_bin_y,la_y_pred) # lasso
fpr_7,tpr_7, thresholds_7 = roc_curve(test_bin_y,best_knn.predict_proba(test_x)[:,1]) # knn
fpr_8,tpr_8, thresholds_8 = roc_curve(test_bin_y,best_gbdt.predict_proba(test_x)[:,1]) # gbdt

In [78]:
plt.figure(figsize=(10, 6), dpi=500)
plt.grid(alpha=0.6, linestyle='--')
colors = ['mediumseagreen', 'steelblue', 'mediumpurple', 'darkorange', 'crimson', 'teal', 'olive', 'maroon']
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')  # navy

plt.plot(fpr_1,tpr_1, color='mediumseagreen',label='LR ROC curve (area = %0.4f)' % lr_auc)
plt.plot(fpr_2,tpr_2, color='steelblue',label='RF ROC curve (area = %0.4f)' % rf_auc)
plt.plot(fpr_3,tpr_3, color='mediumpurple',label='XGB ROC curve (area = %0.4f)' % xgb_auc)
plt.plot(fpr_4,tpr_4, color='darkorange',label='SVM ROC curve (area = %0.4f)' % svm_auc)
plt.plot(fpr_6,tpr_6, color=colors[5],label='LASSO ROC curve (area = %0.4f)' % la_auc)
plt.plot(fpr_7,tpr_7, color=colors[6],label='KNN ROC curve (area = %0.4f)' % knn_auc)


my_x_ticks = np.linspace(0, 1, 11)  # Set x-axis from 0 to 1 with 11 intervals
plt.xticks(my_x_ticks, fontsize=12)
my_y_ticks = np.linspace(0, 1, 11)  # Set y-axis from 0 to 1 with 11 intervals
plt.yticks(my_y_ticks, fontsize=12)

plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC of The Gene classification', fontsize=16, pad=10)
# plt.legend(loc='best')  # "lower right"
plt.legend(loc='best', fontsize=12, frameon=True, fancybox=True, framealpha=1, shadow=True, borderpad=1)
plt.tight_layout()
plt.show()